In [ ]:
from collections.abc import Iterable
import os

from datetime import datetime, timedelta
from dateutil import tz
from hdmf.backends.hdf5.h5_utils import H5DataIO
from hdmf.container import Container
from hdmf.data_utils import DataChunkIterator
import latex
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import numpy as np
import pandas as pd
import pickle
from pynwb import load_namespaces, get_class, register_class, NWBFile, TimeSeries, NWBHDF5IO
from pynwb.file import MultiContainerInterface, NWBContainer, Device, Subject
from pynwb.ophys import ImageSeries, OnePhotonSeries, OpticalChannel, ImageSegmentation, PlaneSegmentation, Fluorescence, DfOverF, CorrectedImageStack, MotionCorrection, RoiResponseSeries, ImagingPlane
from pynwb.core import NWBDataInterface
from pynwb.epoch import TimeIntervals
from pynwb.behavior import SpatialSeries, Position
from pynwb.image import ImageSeries
import pywt
import scipy.io as sio
import scipy
from scipy.stats import multivariate_normal
from scipy.optimize import linear_sum_assignment
import seaborn as sns
import skimage.io as skio
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from tifffile import TiffFile
import tifffile

from atlas import loadmat, NPAtlas, NWBAtlas
from process_file import get_nwb_neurons, get_dataset_neurons, get_dataset_online, combine_datasets, get_pairings, get_color_discrim, get_neur_nums
from stats import get_summary_stats, analyze_pairs, get_accuracy
from visualization import plot_num_heatmap, plot_std_heatmap, plot_summary_stats, plot_color_discrim, plot_accuracies, plot_visualizations_atlas, plot_visualizations_data, plot_atlas2d_super
from utils import covar_to_coord, convert_coordinates, maha_dist, run_linear_assignment

# ndx_mulitchannel_volume is the novel NWB extension for multichannel optophysiology in C. elegans
from ndx_multichannel_volume import CElegansSubject, OpticalChannelReferences, OpticalChannelPlus, ImagingVolume, VolumeSegmentation, MultiChannelVolume, MultiChannelVolumeSeries

import os
import PyQt6.QtCore
os.environ["QT_API"] = "pyqt6"


In [ ]:
import os
import PyQt6.QtCore
os.environ["QT_API"] = "pyqt6"


In [ ]:
filepath = '/Users/danielysprague/foco_lab/data/NP_nwb/56_YAaDV.nwb'
#filepath = '/Users/danielysprague/foco_lab/data/NWB_foco/2021-12-03-w00-NP1.nwb'

with NWBHDF5IO(filepath, mode='r', load_namespaces=True) as io:
    read_nwbfile = io.read()
    #print(read_nwbfile.processing['ProcessedImage'])
    subject = read_nwbfile.subject #get the metadata about the experiment subject
    growth_stage = subject.growth_stage
    image = read_nwbfile.acquisition['NeuroPALImageRaw'].data[:] #get the neuroPAL image as a np array
    channels = read_nwbfile.acquisition['NeuroPALImageRaw'].RGBW_channels[:] #get which channels of the image correspond to which RGBW pseudocolors
    im_vol = read_nwbfile.acquisition['NeuroPALImageRaw'].imaging_volume #get the metadata associated with the imaging acquisition
    seg = read_nwbfile.processing['NeuroPAL']['NeuroPALSegmentation']['NeuroPALNeurons'].voxel_mask[:] #get the locations of neuron centers
    labels = read_nwbfile.processing['NeuroPAL']['NeuroPALSegmentation']['NeuroPALNeurons']['ID_labels'][:]
    optchans = im_vol.optical_channel_plus[:] #get information about all of the optical channels used in acquisition
    chan_refs = read_nwbfile.processing['NeuroPAL']['OpticalChannelRefs'].channels[:] #get the order of the optical channels in the image
    #calcium_frames = read_nwbfile.acquisition['CalciumImageSeries'].data[0:15, :,:,:] #load the first 15 frames of the calcium images
    #print(read_nwbfile.acquisition['CalciumImageSeries'].dimension[:])
    #fluor = read_nwbfile.processing['CalciumActivity']['SignalRawFluor']['SignalCalciumImResponseSeries'].data[:]
    #calc_labels = read_nwbfile.processing['CalciumActivity']['NeuronIDs'].labels[:]

    NIR = read_nwbfile.processing['BF_NIR']['BrightFieldNIR'].data[:,:,:]

read_nwbfile




In [ ]:
ts = fluor.T

ref_ind = np.argwhere(calc_labels=='AVAR')
ref_trace = np.squeeze(ts[ref_ind,:])

to_plot = ref_trace / np.mean(ref_trace)

print(ref_ind)
print(ref_trace.shape)

plt.plot(ref_trace)
plt.show()

In [ ]:
calc_max = np.max(calcium_frames, axis=0)
zmax_calc = np.max(calcium_frames[10,:,:,:], axis=2)
plt.imshow(zmax_calc)
plt.show()
plt.figure()

plt.imshow(NIR[10,:,:])

In [ ]:
blobs = pd.DataFrame.from_records(seg, columns = ['X', 'Y', 'Z', 'weight', 'ID'])
blobs = blobs.drop(['weight'], axis=1)
blobs = blobs.replace('nan', np.nan, regex=True) 

print(image.shape)

RGB = image[:,:,:,channels[:-1]]/np.max(image)

print(RGB.shape)

Zmax = np.max(RGB, axis=2)
Ymax = np.max(RGB, axis=1)

plt.figure()

plt.imshow(np.transpose(Zmax, [1,0,2]))
plt.scatter(blobs['x'], blobs['y'], s=5)
plt.xlim((0, Zmax.shape[0]))
plt.ylim((0, Zmax.shape[1]))
plt.gca().set_aspect('equal')

plt.show()

plt.figure()

plt.imshow(np.transpose(Ymax, [1,0,2]))
plt.scatter(blobs['x'], blobs['z'], s=5)
plt.xlim((0, Ymax.shape[0]))
plt.ylim((0, Ymax.shape[1]))
plt.gca().set_aspect('equal')

plt.show()